In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [2]:
import os

# Establece la GPU a usar (en este caso la GPU #1)
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
#os.environ["CUDA_VISIBLE_DEVICES"] = "'0,1,2,3"

In [3]:
print(os.environ.get("CUDA_VISIBLE_DEVICES"))

1


In [4]:
import tensorflow as tf; print(tf.config.list_physical_devices('GPU'))

2025-02-12 19:38:06.653142: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1739389086.671263 2701900 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1739389086.676937 2701900 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-12 19:38:06.695768: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]


In [4]:
train = pd.read_csv("train.csv")
validation = pd.read_csv("validation.csv")
test = pd.read_csv("test.csv")

In [5]:
#Eliminamos la primera columna, generada al pasar el conjunto de datos a formato csv.
train=train.drop(columns='Unnamed: 0')
validation=validation.drop(columns = 'Unnamed: 0')
test=test.drop(columns='Unnamed: 0')

In [6]:
porcentaje_mantener = 0.0005


train_reducido_udp = train[train['proto'] == 'UDP'].groupby('category').sample(frac=porcentaje_mantener)
train_reducido_tcp = train[train['proto'] == 'TCP'].groupby('category').sample(frac=porcentaje_mantener)

train_reducido = pd.concat([train_reducido_udp, train_reducido_tcp])

validation_reducido_udp = validation[validation['proto'] == 'UDP'].groupby('category').sample(frac=porcentaje_mantener)
validation_reducido_tcp = validation[validation['proto'] == 'TCP'].groupby('category').sample(frac=porcentaje_mantener)

validation_reducido = pd.concat([validation_reducido_udp, validation_reducido_tcp])

test_reducido_udp = test[test['proto'] == 'UDP'].groupby('category').sample(frac=porcentaje_mantener)
test_reducido_tcp = test[test['proto'] == 'TCP'].groupby('category').sample(frac=porcentaje_mantener)

test_reducido = pd.concat([test_reducido_udp, test_reducido_tcp])

In [7]:
train_reducido['category'].value_counts()

category
Benign                 56
DDoS-UDP_Flood         56
DoS-SYN_Flood          56
DoS-HTTP_Flood         56
DoS-UDP_Flood          56
Recon-HostDiscovery    56
DDoS-SYN_Flood         56
DDoS-HTTP_Flood        55
Recon-PortScan         55
Recon-OSScan           55
Name: count, dtype: int64

In [8]:
train_reducido['proto'].replace({'TCP': 0, 'UDP': 1}, inplace=True)
validation_reducido['proto'].replace({'TCP': 0, 'UDP': 1}, inplace=True)
test_reducido['proto'].replace({'TCP': 0, 'UDP': 1}, inplace=True)

/tmp/ipykernel_3045897/732036173.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train_reducido['proto'].replace({'TCP': 0, 'UDP': 1}, inplace=True)
/tmp/ipykernel_3045897/732036173.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train_reducido['proto'].replace({'TCP': 0, 'UDP': 1}, inplace=

In [9]:
# Separar características y etiquetas para conjunto de entrenamiento
X_train_reducido = train_reducido.drop(columns=['category'])
y_train_reducido = train_reducido['category']

# Separar características y etiquetas para conjunto de validación
X_val_reducido = validation_reducido.drop(columns=['category'])
y_val_reducido = validation_reducido['category']

# Separar características y etiquetas para conjunto de prueba
X_test_reducido = test_reducido.drop(columns=['category'])
y_test_reducido = test_reducido['category']

In [10]:
#Normalización del conjunto de datos reducido
norm = StandardScaler()

X_train_reducido = norm.fit_transform(X_train_reducido)
X_val_reducido = norm.fit_transform(X_val_reducido)
X_test_reducido = norm.fit_transform(X_test_reducido)

pca = PCA(n_components=25)
X_train_reducido = pca.fit_transform(X_train_reducido)
X_val_reducido = pca.transform(X_val_reducido)
X_test_reducido = pca.transform(X_test_reducido)

In [11]:
from qiskit_ibm_runtime import Session, SamplerV2, EstimatorV2
from qiskit.providers.fake_provider import Fake127QPulseV1
from qiskit_ibm_runtime.fake_provider import FakeBrisbane 
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_aer import AerSimulator, QasmSimulator
#from qiskit_aer.primitives import SamplerV2

from qiskit_machine_learning.state_fidelities import ComputeUncompute
from qiskit_machine_learning.kernels import FidelityQuantumKernel
from qiskit.circuit.library import ZZFeatureMap

num_qubits = X_train_reducido.shape[1]

backend = AerSimulator(method='statevector', device='GPU')
session = Session(backend=backend)
pass_manager = generate_preset_pass_manager(optimization_level=0, target=backend.target)

sampler = SamplerV2(mode=session)
fidelity = ComputeUncompute(sampler=sampler, pass_manager=pass_manager)

feature_map = ZZFeatureMap(feature_dimension=num_qubits)
qk = FidelityQuantumKernel(feature_map=feature_map, fidelity=fidelity)

In [12]:
# Verificar si se seleccionó GPU
print(f"Backend device: {backend.options.device}")

Backend device: GPU


In [ ]:
from qiskit_machine_learning.algorithms import QSVC
import time

qsvc = QSVC(quantum_kernel=qk)

start = time.time()
qsvc.fit(X_train_reducido, y_train_reducido)
stop = time.time()

train_time = stop - start

#start = time.time()
#qsvc_score = qsvc.score(X_test_reducido, y_test_reducido)
#stop = time.time()

#test_time = stop - start

print(f"Training Time: {train_time}")
#print(f"Testing Time: {test_time}")
#print(f"Total Time: {train_time + test_time}")
#print(f"QSVC classification test score: {qsvc_score}")

In [ ]:
import pandas as pd
import numpy as np
import time
import os
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn import svm
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve
from sklearn import metrics
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc

In [ ]:
y_pred_SVM_val = qsvc.predict(X_val_reducido)
y_pred_SVM_test = qsvc.predict(X_test_reducido)

In [ ]:
cm1 = metrics.confusion_matrix(y_val_reducido, y_pred_SVM_val)
normalized_matrix = cm1 #/ cm.sum(axis=1, keepdims=True)

class_names = ['Benign', 'DDoS-HTTP_Flood', 'DDoS-SYN_Flood', 'DDoS-UDP_Flood', 'DoS-HTTP_Flood', 'DoS-SYN_Flood', 'DoS-UDP_Flood', 'Recon-HostDiscovery', 'Recon-OSScan', 'Recon-PortScan']

sns.heatmap(normalized_matrix, annot = True, fmt = '0.0f', linewidth=0.5, xticklabels=class_names,
            yticklabels=class_names, square = True, cmap='Blues', cbar = False)

fig = plt.gcf()
fig.set_size_inches(5, 5)

plt.ylabel('Valores Reales')
plt.xlabel('Valores Predichos')
plt.show()